### Simulation of the Hong-Page "Diversity Trumps Ability" Argument 

This script recreates the simulation results of the Hong Page (2004) "diversity trumps ability" (DTA) argument for finite populations. 

In [1]:
# Import libraries
import random
import numpy as np
import statistics
import pandas as pd
from IPython.display import display
import time
import itertools
import json

In [2]:
# Inputs:

# In Hong-Page paper, main results are for n = 2,000, l = 12 or 20, k = 3
# They experimented with l varying between 6 and 20, k varying between 2 and 7, and n varying 
# between 200 and 10,000. Within these parameter ranges, they found qualitatively similar phenomena.

# Number of integers in X
n = 100

# 1 <= k < l < n
l = 20
k = 3

In [3]:
# Number of unique heuristics (agents in model) = l*(l-1)*...*(l-k+1) :
lst = [l-i for i in range(0,k)]
n_heuristics = np.prod(lst) 

print('The total number of possible agent heuristics in this model is: ',n_heuristics)

The total number of possible agent heuristics in this model is:  6840


In [4]:
# The value of each of the n points is independently drawn according to the uniform distribution on the interval [0, 100].
X = [random.randint(0,100) for i in range(1,n+1)]

In [5]:
# An agent, alpha, is a list of k distinct integers in [1, . . . , l] which they use to find the maximum value of V.
# An agents heuristics is given by PHI = (PHI1, PHI2, .... ,PHIk) where each PHIi is in the set {1,2,...,l}  

# Find list of all possible combinations given k and l
lists = [list(range(1,l+1))]*k
all_combinations = list(itertools.product(*lists))

# Python code to remove elements which do not have 3 distinct integers 
def Remove(non_distinct):
    final_list = []
    for x in non_distinct:
        if len(x) > len(set(x)):
            pass
        else:
            final_list.append(x)
    
    return final_list

agent_heuristics = Remove(all_combinations)

len(agent_heuristics)

6840

In [6]:
# Agent methodology for maximizing V
    
def max_method(agent_heuristic, starting_point_index_input):
    #starting_point_index = random.randint(0,n-1)
    starting_point_index = starting_point_index_input
    starting_point = X[starting_point_index]
    
    #max_value_index = starting_point_index
    #max_value = starting_point
    
    # Initialize max index, position and score values
    max_index_values = [starting_point_index]
    position_values = [0]
    score = [1]
    
    while 1 in score:
        position = position_values[0]
        
        # Account for new agent heuristic ordering (it doesn't reset to start after every change in new max value)
        agent_heuristic_new = [a for a in agent_heuristic[position:]]
        for a in agent_heuristic[:position]:
            agent_heuristic_new.append(a)
            agent_heuristic = agent_heuristic_new
        
        max_value_index = max_index_values[0]
        max_value = X[max_value_index]
        
        # Clear max index, position and score values
        max_index_values = []
        position_values = []
        position = 0
        score = []
        
        for i in range(0,len(agent_heuristic)):
            position = position + 1
            if max_value_index+agent_heuristic[i] > n-1:
                if X[max_value_index+agent_heuristic[i]-n] < max_value:
                    score.append(0)
                
                else:
                    max_value_index = max_value_index+agent_heuristic[i]-n
                    max_value = X[max_value_index]
                    
                    max_index_values.append(max_value_index)
                    position_values.append(position)
                    score.append(1)
            

            else:
                if X[max_value_index+agent_heuristic[i]] < max_value:
                    score.append(0)
                    
                else:
                    max_value_index = max_value_index+agent_heuristic[i]
                    max_value = X[max_value_index]
                    
                    max_index_values.append(max_value_index)
                    position_values.append(position)
                    score.append(1)
            
        
        max_value = X[max_value_index] 
        
    return max_value_index, max_value

In [7]:
start_time = time.time()

In [8]:
# Calculate each agent's score by the expected value of the stopping points from each possible initial starting point
# It is assumed each possible initial starting point is equally likely to occur

dict_scores = {}

for heuristic in agent_heuristics:
    scores = []
    for z in range(0,n):
        scores.append(max_method(heuristic,z)[-1])
        dict_scores[str(heuristic)] = statistics.mean(scores)



In [9]:
end_time = time.time()
print(f"It took {end_time-start_time:.2f} seconds to run each agent's expected score.") 

It took 36.38 seconds to run each agent's expected score.


In [28]:
# Top 10 best performing agents/heuristics 

# Number assessed can be inputed, here the default is the top 10 individuals 
num_assessed = 10 

#dict1 = {1: 1, 2: 9, 3: 4}
sorted_dict = {}
sorted_keys = sorted(dict_scores, key=dict_scores.get, reverse=True)  # [1, 3, 2]

# Convert string values to array

# ability_group = [json.loads(i) for i in sorted_keys[:num_assessed]]

# JSON not working for some reason so using alternative code:
ability_group = [ [int(x) for x in i[1:-1].split(', ')] for i in sorted_keys[:num_assessed]]


# Initialize data of lists
data = {'Heuristic/Agent':ability_group,
        'Average Value':[dict_scores[i] for i in sorted_keys[:num_assessed]]}
  
# Create DataFrame of top 10 performers
ability_group_df = pd.DataFrame(data)
  
# Print the output.
display(ability_group_df)

,Heuristic/Agent,Average Value
0,"[1, 14, 11]",91.60
1,"[11, 1, 14]",91.59
2,"[7, 9, 14]",91.39
3,"[11, 14, 1]",91.34
4,"[1, 11, 14]",91.33
5,"[6, 14, 2]",91.28
6,"[16, 14, 11]",91.23
7,"[7, 14, 9]",91.16
8,"[9, 7, 14]",91.15
9,"[5, 16, 14]",91.14


In [29]:
# Now we take 10 random individuals from population of heuristics using the sample() method (without replacement)

random_group = random.sample(agent_heuristics, num_assessed)

print(random_group)

[(7, 3, 11), (8, 2, 18), (15, 7, 19), (16, 4, 14), (7, 15, 6), (9, 12, 8), (1, 16, 8), (4, 16, 7), (17, 20, 5), (10, 16, 3)]


In [30]:
# Calculate the diversity score of both groups

# Number of unique paris in population is given by: n(n-1)/2
print('Number of possible unique pairs is: ',str(num_assessed*(num_assessed-1)/2))

# Function which returns subset or r length from n
from itertools import combinations
  
def rSubset(arr, r):
  
    # return list of all subsets of length r
    # to deal with duplicate subsets use 
    # set(list(combinations(arr, r)))
    
    return list(combinations(arr, r))

# Random group diversity score:
random_diversity_scores = []

for i in rSubset(random_group,2):
    a = i[0]
    b = i[-1]
    for x in range(0,len(a)):
        if a[x] != b[x]:
            random_diversity_scores.append(1)
            
random_diversity_score = len(random_diversity_scores)
        
print('Random group diversity score is:',str(random_diversity_score))

# Ability group diversity score:
ability_diversity_scores = []

for i in rSubset(ability_group,2):
    a = i[0]
    b = i[-1]
    for x in range(0,len(a)):
        if a[x] != b[x]:
            ability_diversity_scores.append(1)
            
ability_diversity_score = len(ability_diversity_scores)
        
print('Ability group diversity score is:',str(ability_diversity_score))

Number of possible unique pairs is:  45.0
Random group diversity score is: 130
Ability group diversity score is: 111


In [31]:
# Collective performance of ability agents 

collective_scores = []
for z in range(0,n):
    max_index = z
    
    score = [1]
    
    while 1 in score:
        score = []
        
        for i in range(0,len(ability_group)):
            if max_method(ability_group[i],max_index)[0] == max_index:
                score.append(0)
                pass
            
            else:
                new_max_index = max_method(ability_group[i],max_index)[0]
                max_index = new_max_index 
                score.append(1)
    
    max_value = X[max_index]
    collective_scores.append(max_value)
    
ability_collective_score = round(statistics.mean(collective_scores),2)
print(ability_collective_score)

98.01


In [32]:
# Collective performance of random agents 

collective_scores = []
for z in range(0,n):
    max_index = z
    
    score = [1]
    
    while 1 in score:
        score = []
        
        for i in range(0,len(random_group)):
            if max_method(random_group[i],max_index)[0] == max_index:
                score.append(0)
                pass
            
            else:
                new_max_index = max_method(random_group[i],max_index)[0]
                max_index = new_max_index 
                score.append(1)
                       
    max_value = X[max_index]
    collective_scores.append(max_value)
    
random_collective_score = round(statistics.mean(collective_scores),2)
print(random_collective_score)

98.4


In [33]:
# Summary Table 

# Initialize data of lists
summary_table_data = {'': ['Best Agents','Random Agents'],
                      "Performance (Average Score of Group)":[ability_collective_score,random_collective_score],
                      "Diversity": [ability_diversity_score,random_diversity_score]}

# Create DataFrame of top 10 performers
summary_table_df = pd.DataFrame(summary_table_data)
  
display(summary_table_df)

,,Performance (Average Score of Group),Diversity
0,Best Agents,98.01,111
1,Random Agents,98.40,130


Therefore, the simulation results show that groups of more diverse individuals collectively perform groups of high ability agents (hence the DTA argument). However, in reality this is just an example of the benefits of randomization. Thompson (2014) refutes the DTA argument by showing that the mathematical theorem for infinite populations is incorrect as stated originally and trivial once corrected. While the simulations for finite populations are contrived optimization problems where the restrictions on the algorithms are artificial and simply illustrate the benefits of randomness and not “diversity”, an already well-known fact in computational theory (Mitzenmacher and Upfal, 2005). 

#### References

- Hong, L., Page, S., (2004). “Groups of diverse problem solvers can outperform groups of high-ability problem solvers”. PNAS 101(46):16385–89.
- Mitzenmacher, M., Upfal, E., (2005). “Probability and Computing”. Cambridge University Press.
- Thompson, A., (2014). “Does diversity trump ability? An example of the misuse of mathematics in the social sciences”. Notices AMS 61(9):1024–30.
